In [1]:
from datetime import datetime, timedelta
from confluent_kafka import Consumer, KafkaError
import json

# Kafka consumer configuration
conf = {
    'bootstrap.servers': 'b-1.monstercluster1.6xql65.c3.kafka.eu-west-2.amazonaws.com:9092',  # Replace with your broker URLs
    'group.id': 'your_group_id',
    'auto.offset.reset': 'earliest',
}

ModuleNotFoundError: No module named 'confluent_kafka'

In [ ]:

consumer = Consumer(conf)
consumer.subscribe(['monster-damage'])

# Create an in-memory buffer to hold data for the current window
current_window_start = datetime.now()
current_window_end = current_window_start + timedelta(minutes=10)
window_data = []

def consume_message():
    try:
        while True:
            msg = consumer.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    continue
                else:
                    print(msg.error())
                    break

            # Process the message
            data = json.loads(msg.value().decode('utf-8'))
            process_message(data)

    finally:
        consumer.close()


In [ ]:
def process_message(data):
    global current_window_start, current_window_end, window_data
    timestamp = datetime.strptime(data['ts'], '%Y-%m-%d %H:%M:%S.%f')
    
    if current_window_start <= timestamp < current_window_end:
        window_data.append(data)
    else:
        # Aggregate data for that window
        aggregate_data()
        # Start a new window
        current_window_start = current_window_end
        current_window_end = current_window_start + timedelta(minutes=10)
        window_data = [data]

In [ ]:
def aggregate_data():
    continent_counts = {}
    for entry in window_data:
        continent = entry['continent']
        continent_counts[continent] = continent_counts.get(continent, 0) + 1

    print(f"Monsters from {current_window_start} to {current_window_end}:")
    for continent, count in continent_counts.items():
        print(f"{continent}: {count} monsters")

consume_message()